In [2]:
import numpy as np
import decimal

Since we are regulating a single LiPo cell (4.2V to 3.2V-ish), down to 3.4V, it's actually not that bad of an idea to just use a LDO.

Finding our worst case efficiency (https://toshiba.semicon-storage.com/eu/semiconductor/knowledge/e-learning/basics-of-low-dropout-ldo-regulators/chap4/chap4-1.html)

In [3]:
V_out = 3.4
V_in_max = 4.2

eff = V_out/V_in_max
eff

0.8095238095238094

Even accounting for other losses through the LDO we probably end up with more than 75% efficiency in the worst case.

The LDO will power most of the peripherals other than the Pi.
This includes:
- RP2040 (~55mA)
- QSPI flash (~25mA)
- Clickwheel (???mA)
- Audio DAC (~32mA)
- Audio Amp (~25mA)
- Display (~100mA)
- GPIO expander (max 125mA)

This leaves us with an approximate total power consumption in the range of 300mA during typical operation.
For safety factor, we will target an LDO capable of sourcing 500mA, preferably with less than 200mV dropout.

Some options:
- https://lcsc.com/product-detail/Linear-Voltage-Regulators-LDO_HEERMICR-HE9073A33MR_C723793.html
    - Not currently in stock
- https://lcsc.com/product-detail/Linear-Voltage-Regulators-LDO_Torex-Semicon-XC6210B332PR-G_C484291.html


**Below is code for setting values on the Techcode Semicon TD5830B, which is no longer used**

https://lcsc.com/product-detail/Linear-Voltage-Regulators-LDO_Techcode-Semicon-TD5830B_C232922.html

The formula to set $V_o$ is:
$$V_o = 0.8\text{V} \left(1 + \frac{R_1}{R_2}\right)$$

Let's bruteforce some values to calculate appropriate resistor values.

In [4]:
I_adj = 100e-6  # A, Typical value from datasheet
tol = 50e-3 # V, tolerance for brute force calc

# https://en.wikipedia.org/wiki/E_series_of_preferred_numbers#E24_vs._E48,_E96,_E192
def V_n(m=48):
    if m >= 48:
        sig_figs = 3
    else:
        sig_figs = 2
    n = np.arange(0, m)
    return np.around(10**(n/m), decimals=sig_figs - 1)
R_vals = V_n(48)

def V_o(R_1, R_2):
    return 0.8*(1 + R_1/R_2)

r1_idx = 0
r2_idx = 0
r1_mag = 1000
r2_mag = 1000
r1 = R_vals[r1_idx]*r1_mag
r2 = R_vals[r2_idx]*r2_mag

while True:
    r1 = R_vals[r1_idx]*r1_mag
    r2 = R_vals[r2_idx]*r2_mag
    v_o = V_o(r1, r2)
    print(f"R_1: {r1:.2f}, R_2: {r2:.2f}, V_o: {v_o:.4f}")
    if abs(v_o - V_out) < tol:
        break
    if v_o < V_out:
        r1_idx += 1
        if r1_idx >= len(R_vals):
            r1_idx = 0
            r1_mag *= 10
    else:
        r2_idx += 1
        if r2_idx >= len(R_vals):
            r2_idx = 0
            r2_mag *= 10


R_1: 1000.00, R_2: 1000.00, V_o: 1.6000
R_1: 1050.00, R_2: 1000.00, V_o: 1.6400
R_1: 1100.00, R_2: 1000.00, V_o: 1.6800
R_1: 1150.00, R_2: 1000.00, V_o: 1.7200
R_1: 1210.00, R_2: 1000.00, V_o: 1.7680
R_1: 1270.00, R_2: 1000.00, V_o: 1.8160
R_1: 1330.00, R_2: 1000.00, V_o: 1.8640
R_1: 1400.00, R_2: 1000.00, V_o: 1.9200
R_1: 1470.00, R_2: 1000.00, V_o: 1.9760
R_1: 1540.00, R_2: 1000.00, V_o: 2.0320
R_1: 1620.00, R_2: 1000.00, V_o: 2.0960
R_1: 1690.00, R_2: 1000.00, V_o: 2.1520
R_1: 1780.00, R_2: 1000.00, V_o: 2.2240
R_1: 1870.00, R_2: 1000.00, V_o: 2.2960
R_1: 1960.00, R_2: 1000.00, V_o: 2.3680
R_1: 2050.00, R_2: 1000.00, V_o: 2.4400
R_1: 2150.00, R_2: 1000.00, V_o: 2.5200
R_1: 2260.00, R_2: 1000.00, V_o: 2.6080
R_1: 2370.00, R_2: 1000.00, V_o: 2.6960
R_1: 2490.00, R_2: 1000.00, V_o: 2.7920
R_1: 2610.00, R_2: 1000.00, V_o: 2.8880
R_1: 2740.00, R_2: 1000.00, V_o: 2.9920
R_1: 2870.00, R_2: 1000.00, V_o: 3.0960
R_1: 3010.00, R_2: 1000.00, V_o: 3.2080
R_1: 3160.00, R_2: 1000.00, V_o: 3.3280


In [5]:
# Print values as engineering notation
decimal.Decimal(r1).normalize().to_eng_string(), decimal.Decimal(r2).normalize().to_eng_string()

('5.36E+3', '1.62E+3')